**11.2.1 Architecture and system design**

In [ ]:
# Install necessary libraries
!pip install langchain langchain-openai langchain-community langchain-core sqlalchemy pandas matplotlib numpy pydantic chromadb openai

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import time

# Check if we have an OpenAI API key
import os
if "OPENAI_API_KEY" not in os.environ:
    # If not, prompt for it
    from getpass import getpass
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# Core components and dependencies
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain.prompts import PromptTemplate
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field

# Let's redefine the structured output chain function since the import isn't working
def create_structured_output_chain(pydantic_schema, llm, prompt):
    def _parse_output(output):
        return pydantic_schema.parse_raw(output)

    return prompt | llm | StrOutputParser() | _parse_output

**11.2.2 Setting up the environment**

In [ ]:
def create_sqlite_connection(db_path):
    """Create a connection to a SQLite database."""
    return SQLDatabase.from_uri(f"sqlite:///{db_path}")

def setup_llm(model_name="gpt-3.5-turbo", temperature=0):
    """Initialize the LLM component with appropriate settings."""
    return ChatOpenAI(
        model_name=model_name,
        temperature=temperature,  # Lower temperature for more deterministic outputs
        request_timeout=120       # Extended timeout for complex queries
    )

**11.2.3 Sample database implementation**

In [ ]:
# Create a retail database with customers, products, orders
import sqlite3

def create_retail_database(db_path):
    """
    Create a retail database with customers, products, orders, and order items tables.
    """
    # Connect to SQLite database (will create it if it doesn't exist)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Create customers table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS customers (
        customer_id INTEGER PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        registration_date DATE NOT NULL,
        city TEXT,
        state TEXT,
        lifetime_value REAL
    )
    ''')

    # Create products table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        category TEXT NOT NULL,
        price REAL NOT NULL,
        inventory_count INTEGER NOT NULL,
        description TEXT
    )
    ''')

    # Create orders table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS orders (
        order_id INTEGER PRIMARY KEY,
        customer_id INTEGER NOT NULL,
        order_date DATE NOT NULL,
        total_amount REAL NOT NULL,
        status TEXT NOT NULL,
        FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
    )
    ''')

    # Create order_items table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS order_items (
        order_item_id INTEGER PRIMARY KEY,
        order_id INTEGER NOT NULL,
        product_id INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        price_per_unit REAL NOT NULL,
        FOREIGN KEY (order_id) REFERENCES orders (order_id),
        FOREIGN KEY (product_id) REFERENCES products (product_id)
    )
    ''')

    conn.commit()
    return conn

def populate_sample_data(conn):
    """Populate the database with sample data."""
    cursor = conn.cursor()

    # Sample customers
    customers = [
        (1, 'John', 'Smith', 'john.smith@email.com', '2021-01-15', 'New York', 'NY', 1250.75),
        (2, 'Sarah', 'Johnson', 'sarah.j@email.com', '2021-02-20', 'Los Angeles', 'CA', 890.25),
        (3, 'Michael', 'Brown', 'michael.b@email.com', '2021-03-10', 'Chicago', 'IL', 1475.50),
        (4, 'Emily', 'Davis', 'emily.d@email.com', '2021-04-05', 'Houston', 'TX', 760.80),
        (5, 'David', 'Wilson', 'david.w@email.com', '2021-05-22', 'Phoenix', 'AZ', 2100.30),
        (6, 'Jessica', 'Martinez', 'jessica.m@email.com', '2021-06-18', 'Philadelphia', 'PA', 1350.45),
        (7, 'James', 'Anderson', 'james.a@email.com', '2021-07-30', 'San Antonio', 'TX', 950.20),
        (8, 'Jennifer', 'Taylor', 'jennifer.t@email.com', '2021-08-12', 'San Diego', 'CA', 1680.90),
        (9, 'Robert', 'Thomas', 'robert.t@email.com', '2021-09-25', 'Dallas', 'TX', 720.60),
        (10, 'Lisa', 'Jackson', 'lisa.j@email.com', '2021-10-08', 'San Jose', 'CA', 1890.15)
    ]

    # Sample products
    products = [
        (1, 'Laptop', 'Electronics', 899.99, 25, 'High-performance laptop with 16GB RAM'),
        (2, 'Smartphone', 'Electronics', 699.99, 40, '5G smartphone with 128GB storage'),
        (3, 'Headphones', 'Electronics', 149.99, 60, 'Noise-cancelling wireless headphones'),
        (4, 'Coffee Maker', 'Appliances', 79.99, 30, 'Programmable coffee maker with timer'),
        (5, 'Blender', 'Appliances', 59.99, 25, 'High-speed blender for smoothies and more'),
        (6, 'T-shirt', 'Clothing', 19.99, 100, 'Cotton t-shirt, available in multiple colors'),
        (7, 'Jeans', 'Clothing', 49.99, 75, 'Classic fit jeans, stonewashed'),
        (8, 'Running Shoes', 'Footwear', 89.99, 50, 'Lightweight running shoes with cushioned soles'),
        (9, 'Backpack', 'Accessories', 39.99, 35, 'Water-resistant backpack with laptop compartment'),
        (10, 'Watch', 'Accessories', 129.99, 20, 'Smart watch with fitness tracking')
    ]

    # Sample orders
    orders = [
        (1, 1, '2023-01-05', 949.98, 'Delivered'),
        (2, 2, '2023-01-12', 699.99, 'Delivered'),
        (3, 3, '2023-01-18', 229.98, 'Delivered'),
        (4, 4, '2023-02-03', 79.99, 'Delivered'),
        (5, 5, '2023-02-15', 299.97, 'Delivered'),
        (6, 6, '2023-02-27', 169.97, 'Shipped'),
        (7, 7, '2023-03-10', 199.98, 'Shipped'),
        (8, 8, '2023-03-22', 149.99, 'Processing'),
        (9, 9, '2023-04-05', 89.99, 'Processing'),
        (10, 10, '2023-04-18', 179.98, 'Processing'),
        (11, 1, '2023-05-02', 149.99, 'Delivered'),
        (12, 2, '2023-05-15', 109.98, 'Delivered'),
        (13, 3, '2023-06-01', 899.99, 'Shipped'),
        (14, 4, '2023-06-12', 699.99, 'Processing'),
        (15, 5, '2023-06-25', 169.97, 'Processing')
    ]

    # Sample order items
    order_items = [
        (1, 1, 1, 1, 899.99),
        (2, 1, 3, 1, 49.99),
        (3, 2, 2, 1, 699.99),
        (4, 3, 6, 2, 19.99),
        (5, 3, 9, 1, 39.99),
        (6, 4, 4, 1, 79.99),
        (7, 5, 6, 3, 19.99),
        (8, 5, 7, 1, 49.99),
        (9, 6, 3, 1, 149.99),
        (10, 6, 6, 1, 19.99),
        (11, 7, 8, 1, 89.99),
        (12, 7, 9, 1, 39.99),
        (13, 8, 3, 1, 149.99),
        (14, 9, 8, 1, 89.99),
        (15, 10, 5, 1, 59.99),
        (16, 10, 6, 2, 19.99),
        (17, 11, 3, 1, 149.99),
        (18, 12, 6, 1, 19.99),
        (19, 12, 9, 1, 39.99),
        (20, 13, 1, 1, 899.99),
        (21, 14, 2, 1, 699.99),
        (22, 15, 3, 1, 149.99),
        (23, 15, 6, 1, 19.99)
    ]

    # Insert data
    cursor.executemany('INSERT OR REPLACE INTO customers VALUES (?,?,?,?,?,?,?,?)', customers)
    cursor.executemany('INSERT OR REPLACE INTO products VALUES (?,?,?,?,?,?)', products)
    cursor.executemany('INSERT OR REPLACE INTO orders VALUES (?,?,?,?,?)', orders)
    cursor.executemany('INSERT OR REPLACE INTO order_items VALUES (?,?,?,?,?)', order_items)

    conn.commit()

def create_schema_documentation():
    """Create a well-formatted documentation of the database schema for the LLM."""
    schema_doc = """
    # Retail Database Schema Documentation

    ## Table: customers
    Stores information about customers who have made purchases.

    - customer_id (INTEGER): Primary key, unique identifier for each customer
    - first_name (TEXT): Customer's first name
    - last_name (TEXT): Customer's last name
    - email (TEXT): Customer's email address (unique)
    - registration_date (DATE): Date when customer first registered
    - city (TEXT): Customer's city of residence
    - state (TEXT): Customer's state of residence (2-letter code)
    - lifetime_value (REAL): Total value of all purchases made by customer

    ## Table: products
    Contains details about products available for purchase.

    - product_id (INTEGER): Primary key, unique identifier for each product
    - product_name (TEXT): Name of the product
    - category (TEXT): Product category (e.g., Electronics, Clothing)
    - price (REAL): Current price of the product
    - inventory_count (INTEGER): Number of units in stock
    - description (TEXT): Detailed description of the product

    ## Table: orders
    Records of customer orders.

    - order_id (INTEGER): Primary key, unique identifier for each order
    - customer_id (INTEGER): Foreign key referencing customers table
    - order_date (DATE): Date when the order was placed
    - total_amount (REAL): Total monetary value of the order
    - status (TEXT): Current status of the order (e.g., Processing, Shipped, Delivered)

    ## Table: order_items
    Details of individual items within each order.

    - order_item_id (INTEGER): Primary key, unique identifier for each order item
    - order_id (INTEGER): Foreign key referencing orders table
    - product_id (INTEGER): Foreign key referencing products table
    - quantity (INTEGER): Number of units of the product ordered
    - price_per_unit (REAL): Price of the product at the time of purchase

    ## Relationships
    - A customer can place multiple orders (one-to-many between customers and orders)
    - An order contains multiple order items (one-to-many between orders and order_items)
    - Each order item references exactly one product (many-to-one between order_items and products)
    """

    return schema_doc

# Create and populate the sample database
db_path = 'retail_store.db'
conn = create_retail_database(db_path)
populate_sample_data(conn)
schema_doc = create_schema_documentation()

# Connect to the database using SQLDatabase utility
db = create_sqlite_connection(db_path)

# Print the schema documentation
print(schema_doc)

**11.2.4 Implementation walkthrough**

In [ ]:
# Query understanding and intent classification
class QueryIntent:
    """Class for classifying the intent of a user query."""
    def __init__(self, primary_entity, query_type, required_tables, complexity):
        self.primary_entity = primary_entity
        self.query_type = query_type
        self.required_tables = required_tables
        self.complexity = complexity

def classify_query_intent(query, llm, db_schema):
    """Analyze a natural language query to determine its intent."""

    intent_prompt = PromptTemplate.from_template(
        """You are an expert in SQL and database analysis.
        Given the following database schema and user question, classify the query intent.

        Database Schema:
        {schema}

        User Question:
        {question}

        Provide a structured analysis of the query intent.
        Primary entity: [The main entity being queried]
        Query type: [Type of query (aggregation, filtering, join, simple_retrieval)]
        Required tables: [Comma-separated list of tables needed to answer this query]
        Complexity: [Estimated complexity (simple, medium, complex)]
        """
    )

    chain = intent_prompt | llm
    result = chain.invoke({"schema": db_schema, "question": query})

    # Parse the output using regex
    primary_entity_match = re.search(r"Primary entity: (.+)", result.content)
    query_type_match = re.search(r"Query type: (.+)", result.content)
    tables_match = re.search(r"Required tables: (.+)", result.content)
    complexity_match = re.search(r"Complexity: (.+)", result.content)

    # Extract and process the matches
    primary_entity = primary_entity_match.group(1).strip() if primary_entity_match else "Unknown"
    query_type = query_type_match.group(1).strip() if query_type_match else "Unknown"
    tables_text = tables_match.group(1).strip() if tables_match else "Unknown"
    required_tables = [table.strip() for table in tables_text.split(',')]
    complexity = complexity_match.group(1).strip() if complexity_match else "Unknown"

    return QueryIntent(primary_entity, query_type, required_tables, complexity)

# Converting natural language to SQL
def generate_sql_query(question, db, llm):
    """Generate a SQL query from a natural language question."""
    # Create a prompt template for SQL generation
    sql_prompt = PromptTemplate.from_template(
        """Given the following schema and question, write a SQL query that would answer the question.

        Schema:
        {schema}

        Question: {question}

        SQL Query:"""
    )

    # Create a chain that generates SQL queries
    chain = sql_prompt | llm | StrOutputParser()
    query = chain.invoke({"question": question, "schema": db.get_table_info()})
    return query

# Executing generated SQL queries
def execute_sql_query(query, db):
    """Execute a SQL query against the database with error handling."""
    try:
        result = db.run(query)
        return {"success": True, "result": result, "error": None}
    except Exception as e:
        return {"success": False, "result": None, "error": str(e)}

# Processing results and generating natural language responses
def generate_response(question, sql_query, query_result, llm):
    """Generate a natural language response based on query results."""

    if not query_result["success"]:
        # Handle the error case
        error_prompt = PromptTemplate.from_template(
            """The following SQL query generated from the user's question failed to execute:

            User Question: {question}
            Generated SQL: {sql_query}
            Error: {error}

            Please explain what might have gone wrong in a helpful way:"""
        )

        response = llm.invoke(error_prompt.format(
            question=question,
            sql_query=sql_query,
            error=query_result["error"]
        ))
        return response.content

    # For successful queries, create a response from the results
    result_data = query_result["result"]

    response_prompt = PromptTemplate.from_template(
        """Based on the user's question and the SQL query results, provide a clear,
        natural language response. Include key metrics or insights from the data.

        User Question: {question}
        SQL Query: {sql_query}
        Query Results: {results}

        Natural language response:"""
    )

    response = llm.invoke(response_prompt.format(
        question=question,
        sql_query=sql_query,
        results=result_data
    ))

    return response.content

# End-to-end example
def sql_qa_system(question, db, llm):
    """End-to-end implementation of SQL question-answering system."""

    print(f"User Question: {question}\n")

    # 1. Classify query intent (optional, provides insights about the query)
    intent = classify_query_intent(question, llm, schema_doc)
    print(f"Query Intent Analysis:")
    print(f"- Primary entity: {intent.primary_entity}")
    print(f"- Query type: {intent.query_type}")
    print(f"- Required tables: {', '.join(intent.required_tables)}")
    print(f"- Complexity: {intent.complexity}\n")

    # 2. Generate SQL query
    print("Generating SQL query...")
    sql_query = generate_sql_query(question, db, llm)
    print(f"Generated SQL: {sql_query}\n")

    # 3. Execute the query
    print("Executing query...")
    query_result = execute_sql_query(sql_query, db)
    if query_result["success"]:
        print("Query executed successfully.\n")
        print("Raw results:")
        print(query_result["result"])
        print()
    else:
        print(f"Query failed with error: {query_result['error']}\n")

    # 4. Generate response
    print("Generating natural language response...")
    response = generate_response(question, sql_query, query_result, llm)
    print(f"Final Response: {response}")

    return {
        "question": question,
        "intent": intent,
        "sql_query": sql_query,
        "query_result": query_result,
        "response": response
    }

# Initialize LLM
llm = setup_llm()

# Run an example query
example_question = "How many orders were placed by each customer, and what's their total spend?"
result = sql_qa_system(example_question, db, llm)

**11.2.5 Testing and evaluation**

In [ ]:
# Define test cases
test_cases = [
    {
        "id": 1,
        "question": "What are the top 3 most expensive products?",
        "expected_tables": ["products"],
        "expected_operations": ["ORDER BY", "LIMIT"]
    },
    {
        "id": 2,
        "question": "How many orders did each customer place?",
        "expected_tables": ["customers", "orders"],
        "expected_operations": ["COUNT", "GROUP BY"]
    },
    {
        "id": 3,
        "question": "What is the total revenue from electronics products?",
        "expected_tables": ["products", "order_items"],
        "expected_operations": ["SUM", "WHERE"]
    },
    {
        "id": 4,
        "question": "Which customer spent the most money?",
        "expected_tables": ["customers", "orders"],
        "expected_operations": ["MAX", "ORDER BY", "LIMIT"]
    },
    {
        "id": 5,
        "question": "What's the average order value for orders containing electronics products?",
        "expected_tables": ["orders", "order_items", "products"],
        "expected_operations": ["AVG", "WHERE", "JOIN"]
    }
]

def evaluate_sql_generation(generated_sql, test_case):
    """Evaluate the generated SQL against expected operations and tables."""
    score = 0
    max_score = len(test_case["expected_tables"]) + len(test_case["expected_operations"])

    # Check for expected tables
    for table in test_case["expected_tables"]:
        if table.lower() in generated_sql.lower():
            score += 1
            print(f"✓ Found expected table: {table}")
        else:
            print(f"✗ Missing expected table: {table}")

    # Check for expected operations
    for operation in test_case["expected_operations"]:
        if operation.lower() in generated_sql.lower():
            score += 1
            print(f"✓ Found expected operation: {operation}")
        else:
            print(f"✗ Missing expected operation: {operation}")

    percentage = (score / max_score) * 100
    return {"score": score, "max_score": max_score, "percentage": percentage}

def run_test_suite(test_cases, db, llm):
    """Run the entire test suite and evaluate performance."""
    results = []

    for test_case in test_cases:
        print(f"\n{'='*50}")
        print(f"Running test case {test_case['id']}: {test_case['question']}")
        print(f"{'='*50}")

        start_time = time.time()

        # Generate SQL
        sql_query = generate_sql_query(test_case["question"], db, llm)

        # Evaluate SQL generation
        print("\nEvaluating SQL generation:")
        eval_result = evaluate_sql_generation(sql_query, test_case)

        # Execute the query to check if it runs
        execution_result = execute_sql_query(sql_query, db)
        execution_time = time.time() - start_time

        # Store results
        test_result = {
            "test_id": test_case["id"],
            "question": test_case["question"],
            "generated_sql": sql_query,
            "evaluation": eval_result,
            "execution_success": execution_result["success"],
            "execution_time": execution_time
        }

        if execution_result["success"]:
            print(f"\n✓ Query executed successfully ({execution_time:.2f}s)")
        else:
            print(f"\n✗ Query execution failed: {execution_result['error']} ({execution_time:.2f}s)")

        print(f"Evaluation score: {eval_result['percentage']:.1f}% ({eval_result['score']}/{eval_result['max_score']})")

        results.append(test_result)

    return results

# Run the test suite
print("\nRunning test suite...")
test_results = run_test_suite(test_cases, db, llm)

# Calculate overall performance
total_score = sum(r["evaluation"]["score"] for r in test_results)
total_max = sum(r["evaluation"]["max_score"] for r in test_results)
overall_percentage = (total_score / total_max) * 100

successful_executions = sum(1 for r in test_results if r["execution_success"])
execution_success_rate = (successful_executions / len(test_results)) * 100

avg_execution_time = sum(r["execution_time"] for r in test_results) / len(test_results)

print("\n" + "="*50)
print("Test Suite Results Summary")
print("="*50)
print(f"Overall SQL generation accuracy: {overall_percentage:.1f}%")
print(f"Query execution success rate: {execution_success_rate:.1f}%")
print(f"Average execution time: {avg_execution_time:.2f}s")

# Visualize the results
import matplotlib.pyplot as plt

# Plot SQL generation accuracy by test case
test_ids = [r["test_id"] for r in test_results]
accuracies = [r["evaluation"]["percentage"] for r in test_results]

plt.figure(figsize=(10, 6))
plt.bar(test_ids, accuracies, color='skyblue')
plt.axhline(y=overall_percentage, color='r', linestyle='-', label=f'Average: {overall_percentage:.1f}%')
plt.xlabel('Test Case ID')
plt.ylabel('SQL Generation Accuracy (%)')
plt.title('SQL Generation Accuracy by Test Case')
plt.ylim(0, 100)
plt.legend()
plt.tight_layout()
plt.show()